In [11]:
import numpy as np

# node class
class Node:
    def __init__(self, puzzle, parent=None):
        self.puzzle = puzzle
        self.parent = parent

    # used to calculate g cost
    def find_path(self):
        node, path = self, []

        while node is not None:
            path.append(node.puzzle)
            node = node.parent

        return list(reversed(path))

# priority queue class for keeping track of visited nodes and f costs
class Queue:
    def __init__(self):
        self.nodes = []
        self.fcosts = []

    def pop(self):
        i = self.fcosts.index(min(self.fcosts))
        self.fcosts.pop(i)
        return self.nodes.pop(i)

    def push(self, node, fcost):
        self.nodes.append(node)
        self.fcosts.append(fcost)

    # get f cost of path to node
    def get_fcost(self, child):
        return self.fcosts[self.nodes.index(child)]

    def delete(self, child):
        i = self.nodes.index(child)
        del self.fcosts[i]
        del self.nodes[i]

# to initialize the puzzle and keep track of moves
class Puzzle:
    # initialize the puzzle and find the blank spot
    def __init__(self, puzzle):
        self.puzzle = puzzle

        zero = []
        for i in range(4):
            for j in range(4):
                if self.puzzle[i][j] == 0:
                    zero = [i, j]
        self.blank = zero

    # override default equals method
    def __eq__(self, other):
        return (self.puzzle == other.puzzle).all()
    
    # # for pretty print
    def __repr__(self):
        return "{}".format(self.puzzle)

    # 'swap' the blank tile with another depending on direction passed
    def move(self, direction):
        x, y = self.blank

        # swap right
        if direction == 1 and direction in self.moves():
            self.puzzle[x][y] = self.puzzle[x][y + 1] 
            self.puzzle[x][y + 1] = 0
            self.blank[1] = self.blank[1] + 1

        # swap up
        elif direction == 2 and direction in self.moves():
            self.puzzle[x][y] = self.puzzle[x - 1][y]
            self.puzzle[x - 1][y] = 0
            self.blank[0] = self.blank[0] - 1

        # swap left
        elif direction == 3 and direction in self.moves():
            self.puzzle[x][y] = self.puzzle[x][y - 1]
            self.puzzle[x][y - 1] = 0
            self.blank[1] = self.blank[1] - 1

        # swap down
        elif direction == 4 and direction in self.moves():
            self.puzzle[x][y] = self.puzzle[x + 1][y]
            self.puzzle[x + 1][y] = 0
            self.blank[0] = self.blank[0] + 1

    # determine what moves are valid from the position of the blank
    def moves(self):
        moves = []
        if self.blank[1] < 3: 
            moves.append(1) # can move right
        if self.blank[0] > 0:
            moves.append(2) # can move up
        if self.blank[1] > 0:
            moves.append(3) # can move left
        if self.blank[0] < 3:
            moves.append(4) # can move down

        return moves

    # return the current state of the puzzle
    def get_state(self):
        return self.puzzle

    # function to check if the generated puzzle can be solved
    def is_solvable(self):
        # find row location of 0
        p1, loc = self.puzzle, 0
        for i in range(4):
            for j in range(4):
                if p1[i][j] == 0:
                    loc = 4 - i

        # find number of inversions
        p2 = self.puzzle[self.puzzle != 0]
        inv = 0
        for i, x in enumerate(p2):
            for y in p2[i+1:]:
                if x > y:
                    inv += 1

        return (loc % 2 == 0) != (inv % 2 == 0)
    
    # check to see if the puzzle is in the solved state
    def is_solved(self):
        solved = Puzzle(np.array([[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12], [13, 14, 15, 0]]))
        return (self.puzzle == solved.puzzle).all()

    # step through the next possible path
    def step(self, direction):
        p = Puzzle(self.puzzle.copy())
        p.move(direction)
        return p

# manhattan distance for h cost
def manhattan(puzzle):
    tile_locations = {
        1: [0, 0],
        2: [0, 1],
        3: [0, 2],
        4: [0, 3],
        5: [1, 0],
        6: [1, 1],
        7: [1, 2],
        8: [1, 3],
        9: [2, 0],
        10: [2, 1],
        11: [2, 2],
        12: [2, 3],
        13: [3, 0],
        14: [3, 1],
        15: [3, 2],
        0: [3, 3]
    }    
    p = puzzle.get_state()
    h = 0
    for x in range(4):
        for y in range(4):
            x_goal, y_goal = tile_locations.get(p[x][y])
            h += (abs(x-x_goal) + abs(y-y_goal))
    return h

# a star algorithm
def a_star(puzzle):
    q = Queue()
    node = Node(puzzle)
    visited = []

    # check if puzzle is solvable 
    if not node.puzzle.is_solvable():
        print('NO SOLUTION')
        return

    # push the node and its f cost to the queue
    q.push(node, manhattan(node.puzzle) + 1)

    # while there are nodes in the queue
    while q.nodes != []:
        node = q.pop() # this has the lowest cost, hence priority queue

        # if the puzzle is solved already, stop
        if node.puzzle.is_solved():
            print('SOLUTION FOUND. # OF STEPS NEEDED: ', len(visited))
            print("{}".format(node.puzzle))
            return
        
        for m in node.puzzle.moves():
            child = Node(node.puzzle.step(m), node) # create a new child node

            # if the child is not in the queue and has not been visited yet
            if (not child in q.nodes) and (child.puzzle not in visited):
                # put the child and its f cost in the queue
                q.push(child, manhattan(child.puzzle) + len(child.find_path()) - 1)

            # if the child in in the queue
            elif child in q.nodes:
                # calculate the f cost of this path
                fcost = manhattan(child.puzzle) + len(child.find_path()) - 1

                # if the newer cost is less, replace the child and its cost
                if fcost < q.get_fcost(child):
                    q.delete(child)
                    q.push(child, fcost)

        # append the child to the visited list
        visited.append(node.puzzle)

    print('NO SOLUTION')
    return

# function to generate a random 15 puzzle
def generate_puzzle():
    puzzle = np.random.choice(np.arange(16), size=16, replace=False)
    puzzle = puzzle.reshape(4, 4)
    return puzzle

# puzzle = Puzzle(generate_puzzle())

# going to use a simple puzzle for grading purposes, so that this
# doesn't take forever to run

puzzle = Puzzle(np.array([[2, 3, 4, 8], [1, 6, 7, 0], [5, 10, 11, 12], [9, 13, 14, 15]]))
a_star(puzzle)

SOLUTION FOUND. # OF STEPS NEEDED:  16
[[ 1  2  3  4]
 [ 5  6  7  8]
 [ 9 10 11 12]
 [13 14 15  0]]
